In [1]:
import numpy as np
import pandas as pd
import re
import jieba

In [2]:
# 停去词
stopword_path = "cn_stopwords.txt"
def load_stopwords(file_path):
    stop_words = []
    with open(file_path,encoding='UTF-8') as words:
        stop_words.extend([i.strip() for i in words.readlines()]) 
    return stop_words

# 去掉{% %}之间的内容
def clearContentWithSpecialCharacter(content):
    content = content.replace("{%","1")
    content = content.replace("%}","1")
    pattern = re.compile(r'(1)(.*)(1)')
    return pattern.sub(r'',content)

# 预处理
def review_to_text(review):
    stop_words = load_stopwords(stopword_path)
    
    #去掉@用户名
    review = re.sub("(@[a-zA-Z0-9_\u4E00-\u9FA5]+)",'',review) 

    #去除英文
    review = re.sub("[^\u4e00-\u9fa5^a-z^A-Z]",'',review) 

    # 分词    
    review_cut = jieba.cut(review)
    
    # 去掉停用词
    if stop_words:
        all_stop_words = set(stop_words)
        words = [w for w in review_cut if w not in all_stop_words]
    return words

In [3]:
# 加载语料
def load_curpus(path):
    data = []
    with open(path,"r",encoding="utf-8") as f:
        for line in f:
            [_,sentiment,comment] = line.split(",",2)
            data.append((comment,int(sentiment)))
    return data

# 处理语料
def pro_curpus(path):
    data = []
    with open(path,"r",encoding="utf-8") as f:
        for line in f:
            [_,sentiment,comment] = line.split(",",2) #以，为分界  分成3份
            content = review_to_text(comment)
            data.append((content,int(sentiment)))
    return data

In [4]:
train_data = load_curpus("weibo2018-master/train.txt")
test_data = load_curpus("weibo2018-master/test.txt")

train_data_pro = pro_curpus("weibo2018-master/train.txt")
test_data_pro = pro_curpus("weibo2018-master/test.txt")

train_df = pd.DataFrame(train_data_pro,columns=['content','sentiment'])
test_df = pd.DataFrame(test_data_pro,columns=['content','sentiment'])


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.993 seconds.
Prefix dict has been built successfully.


In [5]:
# 将文档处理成 tokens 
data_str = [' '.join(content) for content,sentiment in train_data_pro ] + \
        [' '.join(content) for content,sentiment in test_data_pro ] # 以' '将切割的词连接(join)起来

In [6]:
# 词向量（数据处理） CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_df = 0.8,
                      min_df = 5)
# CountVectorizer方法常用的参数：
# max_df：在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
# min_df：在低于这一数量的文档中出现的关键词（过于独特），去除掉。
# token_pattern：主要是通过正则处理掉数字和标点符号。
# stop_words：设置停用词表，这样的词我们就不会统计出来（多半是虚拟词，冠词等等），需要列表结构，所以代码中定义了一个函数来处理停用词表。

In [7]:
# 训练模型，朴素贝叶斯算法
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
nb = BernoulliNB()

vect.fit_transform(data_str) # 对训练集用fit_transform 测试集用transform，否则词向量的维度会不等

<10500x5452 sparse matrix of type '<class 'numpy.int64'>'
	with 122536 stored elements in Compressed Sparse Row format>

In [8]:
# 生成训练集 测试集的 词向量
X_data, y_data = [], []
for content, sentiment in train_data_pro:
    X_data.append(" ".join(content))
    y_data.append(sentiment)
X_train = vect.transform(X_data)
y_train = y_data

X_data2,y_data2 =[],[]
for content,sentiment in test_data_pro:
    X_data2.append(" ".join(content))
    y_data2.append(sentiment)
X_test = vect.transform(X_data2)
y_test = y_data2

In [9]:
# 使用贝叶斯进行训练
nb.fit(X_train,y_train)
train_score = nb.score(X_train,y_train)
print(train_score)

0.8748


In [10]:
# 打印 bad_cases
bad_cases = []
for i in range(X_train.shape[0]):
    if(nb.predict(X_train[i])!=y_train[i]):
#         print("[%s],[%s],[真实：%s]，[判断：%s]" %(review_data[i],X_train[i],y_train[i],nb.predict(X_train_vect[i])))
        print("[%s],[%s],[真实：%s]，[判断：%s]" %(train_data[i],X_data[i],y_train[i],nb.predict(X_train[i])))


[('最近又开始听古风。我早就没有了当时那种狂热，现在这个圈子也实在让人爱不起来，喜欢的不过还是那些人罢了，那些歌带来的触动是不变的真实的。那么多人来过又离开，那么多名字被人提及又忘记，其实都只是来自五湖四海的陌生人为了同一个梦想齐聚一堂。是惊才绝艳的年轻人美好却遗憾地相逢。但这个梦想会延续。从前的美好时光，不必太过挂怀。#墨明棋妙村歌十周年版[音乐]#\n', 1)],[最近 听 古风 早就 没有 当时 那种 狂热 现在 圈子 实在 爱 起来 喜欢 歌 带来 触动 不变 真实 多人来 离开 名字 提及 忘记 其实 五湖四海 陌生人 同一个 梦想 齐聚一堂 惊才绝艳 年轻人 美好 遗憾 相逢 梦想 会 延续 从前 美好时光 不必 太过 挂怀 墨明 棋妙 村歌 十周年 版 音乐],[真实：1]，[判断：[0]]
[('@PGONE工作室 麻烦动动手删一下评论。你老板小手手都累坏了，你勤快点，谢谢。 \u200b\n', 0)],[麻烦 动动手 删 一下 评论 老板 小手 手 累坏 勤快 点 谢谢],[真实：0]，[判断：[1]]
[('{%#曝光台#%} 【路边鲜花遭了“毒手”】小编收到了一位市民提供的图片，图片虽不是特别清晰，但也可以看见，这位女士在摘绿化带内的鲜花！[怒] 要知道，这些鲜花此时的美，倾注了绿化管养师傅们平时多少个日夜的精心呵护！小编希望大家，赏花莫“伤花”，争做文明南京人。@南京市绿化园林局 @建邺播报 {%南京·虹悦城%}\n', 0)],[曝光 台 路边 鲜花 遭 毒手 小编 收到 一位 市民 提供 图片 图片 特别 清晰 看见 这位 女士 摘 绿化带 鲜花 怒要 知道 鲜花 美 倾注 绿化 管养 师傅 平时 日夜 精心 呵护 小编 希望 赏花 莫伤 花 争做 文明 南京 南京 虹悦城],[真实：0]，[判断：[1]]
[('举报这条微博，吗的不听劝删，代别人大名上广场，让易烊千玺被骂！太不要脸了，帮别家虐粉拿易烊千玺当剑使！→{%网页链接%} \u200b\n', 0)],[举报 这条 微博 听 劝 删代 大名 广场 易 烊 千玺 骂 太 不要脸 帮 别家 虐粉 拿易 烊 千玺 当剑 网页 链接],[真实：0]，[判断：[1]]
[('每次别人说喜欢我我都觉得他们的喜欢很浅浅到喝过酒就忘记吵过架就放弃 \u200b\n', 0)],[每次 说

[('看到一坨黑黑的，结果是我们豆豆背对我坐着，看着我看他起来转了一个身，面对我坐下了，还对我抖了抖腿，打了一个哈气[允悲]这大爷的坐姿我也是服气[喵喵] {%杭州·西溪（商圈）%} \u200b\n', 1)],[看到 一坨 黑黑的 豆豆 背对 坐 看着 起来 转 一个身 面对 坐下 抖 抖 腿 一个 哈气 允悲 大爷 坐姿 服气 喵 喵 杭州 西溪 商圈],[真实：1]，[判断：[0]]
[('我也曾介意很多事情 介意爱的多面和分割 介意许诺的而不兑现 介意付出的而换不来对等 介意品性的善良和各种表象 时至今日我终于知道 我要求纯粹和珍视的用心真的才是最可笑的 只有真的学会选择放弃一些东西的时候 才能变得更快乐吧[兔子] \u200b\n', 1)],[介意 很多 事情 介意 爱 多面 分割 介意 许诺 兑现 介意 付出 换 不来 介意 品性 善良 表象 时至今日 终于 知道 要求 纯粹 珍视 用心 真的 可笑 真的 学会 选择 放弃 东西 变得 更 快乐 兔子],[真实：1]，[判断：[0]]
[('想说老子绝对不搞了然后被女孩纸们逼着看101正常排完雷就记住了陈芳语和yamy当然华的团还没有出来啦不过最大的感受就是我更爱朱正廷先生了你能出道真好[心]女孩纸们叫我投票，投票是不可能了这辈子都不可能了。老子这辈子就为朱正廷一个人日肝****嘻嘻嘻，baebae咋还没回国啊[中国赞] \u200b\n', 1)],[想 说 老子 绝对 搞 女孩 纸们 逼 正常 排 完雷 记住 陈芳语 yamy 华 团 没有 最大 感受 更 爱 朱正廷 先生 出道 真 好心 女孩 纸们 投票 投票 可能 这辈子 可能 老子 这辈子 朱正廷 一个 人日 肝 嘻嘻 baebae 没 回国 中国 赞],[真实：1]，[判断：[0]]
[('我竟然被一个女生撩的脸红了[允悲][允悲][允悲] \u200b\n', 1)],[竟然 一个 女生 撩 脸红 允 悲允 悲允悲],[真实：1]，[判断：[0]]
[('可笑死我了！可不可以不要乱学我们四川人用叠词啊！我们有睡歪歪搓夹夹但是没有学累累这种说法[笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry][笑cry] \u200b\n

[('这个周从周四就废掉了，小哥哥过来找我，也没心思看书了，每天就想着吃吃吃，玩玩玩，现在是时候收心了，两个人一起去努力，然后最好以后就可以不再继续异地恋咯，期待四十天之后的三周年纪念。 \u200b\n', 1)],[周从 周四 废掉 哥哥 过来 找 没 心思 看书 每天 想着 吃 吃 吃玩 玩玩 现在 收心 两个 一起 努力 最好 以后 不再 继续 异地 恋咯 期待 四十天 之后 三 周年纪念],[真实：1]，[判断：[0]]
[('当有点内向的滴滴司机突然问了我一句“你玩抖音吗”之后，滔滔不绝跟我聊了一路抖音。车上音响放的全是抖音里的音乐。他说每晚十二点上床开始刷抖音，一不留神就凌晨三点了。言语中按捺不住的兴奋。我也出于好奇刷过两天抖音，直观感受是一种没心没肺的欢乐气场。能让大众喜爱到这种程度真的蛮了不起 \u200b\n', 1)],[有点 内向 滴滴 司机 突然 问 一句 玩 抖音 之后 滔滔不绝 聊 一路 抖音 车上 音响 放 全是 抖 音里 音乐 说 每晚 十二点 上床 刷抖音 一不留神 凌晨 三点 言语 中 按捺不住 兴奋 好奇 刷过 两天 抖音 直观 感受 一种 没心没肺 欢乐 气场 大众 喜爱 这种 程度 真的 蛮 了不起],[真实：1]，[判断：[0]]
[("暂且先认为前段时间疯狂熬夜行为是导致我长肉的最大影响因素，那闲下来就应该认真对待这个历史遗留问题。结果在下定决心的第一天晚上'第二天晚上就承受了内心巨大的谴责，但我还是觉得生命不息，希望不灭！明天又是一个重新的开始。 {%天津·咸水沽镇%} \u200b\n", 1)],[暂且 先 认为 前段时间 疯狂 熬夜 行为 导致 我长 肉 最大 影响 因素 闲下来 应该 认真对待 历史 遗留问题 下定决心 第一天 晚上 第二天 晚上 承受 内心 巨大 谴责 觉得 生命不息 希望 灭 明天 一个 重新 天津 咸水 沽 镇],[真实：1]，[判断：[0]]
[('团团这一天都对我很友好猫是不是被人下了药了 \u200b\n', 1)],[团团 一天 友好 猫 是不是 药],[真实：1]，[判断：[0]]
[('一边玩着手机 一边不禁思考   明天明明是星期一为什么都快12点了我还不去洗洗睡后来想想，不对------为什么我不早点洗了躺在床上玩手机[笑cry] \u200b\n', 1)],[一边 玩

[('我的英语课代表Coco（男）喜欢一班的英语课代表，每次一班英语课代表进办公室问作业，他也跟进来假装问我作业（我明明已经在班里布置了）我：一脸鄙夷 来干嘛～Coco：我来瞻仰你…我：可拉倒吧……他一不听话我就说撤掉他的职务，他就特乖。生怕失去进办公室的机会……学生时期的这种我就想远远看着你暗恋你的感觉还挺美好的…\n', 1)],[英语课 代表 Coco 男 喜欢 一班 英语课 代表 每次 一班 英语课 代表 进 办公室 问 作业 进来 假装 问 作业 明明 已经 班里 布置 一脸 鄙夷 干嘛 Coco 我来 瞻仰 拉倒 不听话 说 撤掉 职务 特乖 生怕 失去 进 办公室 机会 学生 时期 这种 想 远远 看着 暗恋 感觉 挺 美好],[真实：1]，[判断：[0]]
[('想让朋友把借的钱今年还给你，难如登天。鼻翼为面相中的财库，鼻翼肥厚易存储钱财，但微漏的就属于进财容易出财更快的类型，而大眼漏神，有下三白，主易遇变故，如车祸、抢劫钱财等事伤身，多表现在35-39岁，破财亦是消灾。另外，面圆鼻弱，夫运不好，操心钱财借款的多是自己，夫妻宫明显凹陷，男女情感互动少，易争吵，建议多想想开心快乐的事情，缓解心态，劳逸结合[心][心]\n', 1)],[想 朋友 钱 今年 还给 难如登天 鼻翼 面相 中 财库 鼻翼 肥厚 易 存储 钱财 微漏 属于 进财 容易 出财 更 快 类型 而大眼 漏神 下三白 主易遇 变故 车祸 抢劫 钱财 事 伤身 表现 岁 破财 消灾 面圆 鼻弱 夫运 不好 操心 钱财 借款 夫妻 宫 明显 凹陷 男女 情感 互动 少易 争吵 建议 想想 开心 快乐 事情 缓解 心态 劳逸结合 心 心],[真实：1]，[判断：[0]]
[('其实真正乐观积极的人，是不需要整天喊着正能量的口号，来说服自己、标榜自己的吧。用正能量简单粗暴地扼杀认知自己的情绪、拥抱自我的机会，也只是太肤浅的表现。却也往往是这些人是‘思想上的巨人，行动上的矮子’。{%#正能量#%} {%Lonely-patients杨先生的秒拍视频%} \u200b\n', 0)],[其实 真正 乐观 积极 需要 整天 喊 正 能量 口号 说服 标榜 正 能量 简单 粗暴 扼杀 认知 情绪 拥抱 自我 机会 太 肤浅 表现 往往 思想 巨人 行动 矮子 正 能量 Lonelypatients 杨

[('生活就是爱这样给你耳光 \u200b\n', 0)],[生活 爱 耳光],[真实：0]，[判断：[1]]
[('希望爱你的人都是善良的[心][心][心]同时希望你能好好的[心][心][心]真的很心疼！不知道你看到这些信息会是多么的难过！ \u200b\u200b\u200b网络暴力真的很可怕！@jjjustin0219 @jjjustin0219 @jjjustin0219 \u200b\n', 1)],[希望 爱 善良 心心心 希望 好好 心心心 真的 心疼 知道 看到 信息 会 难过 网络 暴力 真的 可怕],[真实：1]，[判断：[0]]
[('{%#电影后来的我们#%}后来的我们是否学会了如何去爱？愿后来 我们什么都有了也能拥有着彼此愿后来所有的一切都跟你有关。 \u200b\n', 0)],[电影 后来 后来 是否 学会 爱愿 后来 拥有 愿 后来],[真实：0]，[判断：[1]]
[('“我坚决维护你拥有讨厌别人爱豆情绪的权利，但是我强烈谴责你因此产生的恶毒语言和粗俗行为。” \u200b\n', 1)],[坚决 维护 拥有 讨厌 爱豆 情绪 权利 强烈 谴责 产生 恶毒 语言 粗俗 行为],[真实：1]，[判断：[0]]
[('▽  “别像个怨妇一样到处挑刺   不清楚的事情不要评论   与你无关的事情不要抨击   三观正一点 爱你的人才多一点 \u200b\u200b\u200b\u200b” \u200b\n', 1)],[别像 怨妇 到处 挑刺 清楚 事情 不要 评论 无关 事情 不要 抨击 三观 正 一点 爱 人才 多一点],[真实：1]，[判断：[0]]
[('我觉得德善让我开始重新审视起了自己的价值，一直以为自己没资格被爱来着，一直以为自己什么都做不好来着，突然发现，能让身边的人幸福，开心，已经是一件非常有价值的事情了。不自以为是，亦不妄自菲薄。这句话我一直记在脑子里，心却时时忘记。请回答1988，可真是治愈呀。[心][心][心] \u200b\n', 1)],[觉得 德善 重新 审视 价值 一直 没 资格 爱来 一直 做 不好 突然 发现 身边 幸福 开心 已经 一件 非常 价值 事情 自以为是 妄自菲薄 这句 话 一直 记在 脑子里 心 时时 忘记 请 回答 真是 治愈 心心心],[真实：1]，[判断：[0]]
[('我好喜欢我b

...

[('想真正理解价值投资的真谛，来创业吧。自己创业搞过公司的基本都能大概明白啥是好公司，绝大多数基金经理，研究员始终是纸上谈兵，很难真正理解价值的。。其实，不少价值投资做的好的大牛，都是做实业出身，比如段永平。 \u200b\n', 1)],[想 真正 理解 价值 投资 真谛 创业 创业 搞 公司 基本 大概 明白 公司 绝大多数 基金 经理 研究员 始终 纸上谈兵 很难 真正 理解 价值 其实 不少 价值 投资 做 大牛 做 实业 出身 段 永平],[真实：1]，[判断：[0]]
[('你住的城市下雨了，很想问你有没有带伞。可是我忍住了，因为我怕你说没带， 而我又无能为力， 就像是我爱你， 却给不到你想要的陪伴。 \u200b\u200b\u200b\u200b\n', 0)],[住 城市 下雨 想 问 有没有 带伞 忍住 我怕 说 没带 无能为力 像是 我爱你 不到 想要 陪伴],[真实：0]，[判断：[1]]
[('适合无聊的人可以来一家家看… {%香港特区·香港四季酒店%} \u200b\n', 0)],[适合 无聊 一家家 香港特区 香港 四季 酒店],[真实：0]，[判断：[1]]
[('妈的我的毕设 \u200b\n', 0)],[妈 毕设],[真实：0]，[判断：[1]]
[('{%#四月，再见#%} 突然天就热了，风就暖了，阳光就开始晒了，人也开始困乏了然而，我还没瘦。马上五月十三马拉松了，我却越跑越吃力。马上五月了，这一年才觉开始呢！ \u200b\n', 0)],[四月 再见 突然 天 热 风 暖 阳光 晒 困乏 没 瘦 马上 五月 十三 马拉松 越 跑 越 吃力 马上 五月 一年 觉],[真实：0]，[判断：[1]]
[('和狗哥讨论Sonny Angel，我发现自从我生病之后，狗哥的求生欲真的是越来越强了。 \u200b\n', 1)],[狗哥 讨论 SonnyAngel 发现 生病 之后 狗哥 求生欲 真的 越来越 强],[真实：1]，[判断：[0]]
[('穿着我的脏脏👟跑矿场～这个🍄也不赖，虽然没吃到好吃的白云烤红薯，但也足够弥补我一早在公交车摔个大跟头以及在矿场吹一中午的失落内心～[偷笑][偷笑][偷笑] \u200b\n', 0)],[穿着 脏脏 跑 矿场 不赖 没 吃 好吃 白云 烤红薯 足够 弥补 一早 公交车 摔个 跟头 矿

[('呵呵  当局者迷，旁观者清 \u200b\n', 0)],[当局者迷 旁观者清],[真实：0]，[判断：[1]]
[('呵呵了 {%找不到昵称就算了吧的秒拍视频%} \u200b\n', 0)],[找 不到 昵称 秒 拍 视频],[真实：0]，[判断：[1]]
[('呵呵，十点半到现在[微笑] {%佛山·罗村街区%} {%今生只爱打酱油的秒拍视频%} \u200b\n', 0)],[十点 半到 现在 微笑 佛山 罗村 街区 今生 爱 酱油 秒 拍 视频],[真实：0]，[判断：[1]]
[('唉 不明白了 他怎么那么可爱？还很好看？什么天仙？把自己的纹身贴贴在心脏那边 还羞羞的半遮半掩哈哈哈哈哈哈哈mmp我心跳都漏拍了 \u200b\n', 0)],[明白 可爱 好看 天仙 纹身 贴 贴 心脏 羞羞 半遮半掩 哈哈哈哈 哈哈哈 mmp 心跳 漏 拍],[真实：0]，[判断：[1]]
[('小猪佩奇：你吹你MMP你吹 UP主: 鸡鸡·夫斯基{%#bilibili#%} {%网页链接%} 哈哈哈哈哈哈哈就没有我苏西不会吹的曲子 \u200b\n', 0)],[小猪 佩奇 吹 MMP 吹 UP 主 鸡鸡 夫斯基 bilibili 网页 链接 哈哈哈哈 哈哈哈 没有 苏西 不会 吹 曲子],[真实：0]，[判断：[1]]
[('平安世界，有你同行！我和象拔蚌带我走成为了亲友~\\\\(≧▽≦)/~一起来见证我们的友谊吧！ mmp啊 \u200b\n', 0)],[平安 世界 同行 象拔蚌 带 走 成为 亲友 起来 见证 友谊 mmp],[真实：0]，[判断：[1]]
[('学校突然放起了EIEImmp 又让我哭。 \u200b\n', 0)],[学校 突然 放起 EIEImmp 哭],[真实：0]，[判断：[1]]
[('谁还没有在青春年少时，爱过一个渣男 {%淄博·泉城欧乐堡%} \u200b\n', 0)],[没有 青春年少 时爱过 一个 渣 男 淄博 泉城 欧乐堡],[真实：0]，[判断：[1]]
[('渣男请都走开! \u200b\n', 0)],[渣 男 请 走开],[真实：0]，[判断：[1]]
[('对啊女生都喜欢渣男男生都喜欢绿茶婊所以不！是！因！为！我！园！[跪了] \u200b\n', 0)],[女生 喜欢 渣 男 男生 喜欢 绿茶 婊 我园 跪]

[('“如果他爱你，他就会包容你的一切自私，虚荣和坏脾气。”我才不信呢，爱应当被尊重，我舍不得爱我的人受一点委屈，我要让自己变得足够好，才不辜负他给我的爱。午安。另外乖巧的问一句你们是怎么发现我的微博的(⁎⁍̴̛ᴗ⁍̴̛⁎)认真脸 \u200b\n', 1)],[爱 会 包容 自私 虚荣 坏脾气 不信 爱 应当 尊重 舍不得 爱 受 一点 委屈 我要 变得 足够 辜负 爱 午安 乖巧 问 一句 发现 微博 认真 脸],[真实：1]，[判断：[0]]
[('过痛过哭过，然后笑笑，跟自己说，不要委屈，不要难过，这就是生活。不管遇到了什么烦心事，都不要自己为难自己，过了今天，明天自然会来。 \u200b\n', 1)],[过痛 哭 笑笑 说 不要 委屈 不要 难过 生活 遇到 烦心事 不要 为难 今天 明天 自然 会],[真实：1]，[判断：[0]]
[('凡事都不能委屈自己🐳(第一波) \u200b\n', 1)],[凡事 不能 委屈 第一波],[真实：1]，[判断：[0]]
[('“不生气了。”像小孩子丢失了心爱的玩具被哄好了……有种委屈无奈又宠溺的感觉！！！呜呜呜他真的是太好了啊……{%#李泽言#%} \u200b\n', 1)],[生气 小孩子 丢失 心爱 玩具 哄 有种 委屈 无奈 宠 溺 感觉 呜呜 真的 太好了 李泽言],[真实：1]，[判断：[0]]
[('我没有任何的理由去委屈我自己，谢谢亲爱的@小钢牙大脸蛋 ，当和你聊完天以后，我突然感觉我从自己的糟糕的状态里出来了，好像一下子释然了。。。[爱你] \u200b\n', 1)],[没有 理由 委屈 谢谢 亲爱 聊 完天 以后 突然 感觉 糟糕 状态 里 好像 一下子 释然 爱],[真实：1]，[判断：[0]]
[('{%#敖子逸#%} 小逸，麻麻5月12日有一场考试如果在此之前公司还没有给出一个合理的解释，我跑到公司楼下拉横幅！我放在心尖尖上的宝贝不能莫名其妙的就受了委屈！ \u200b\n', 1)],[敖子逸小逸 麻麻 月 日 一场 考试 在此之前 公司 没有 给出 一个 合理 解释 跑 公司 楼下 拉 横幅 放在 心尖 尖上 宝贝 不能 莫名其妙 受 委屈],[真实：1]，[判断：[0]]
[('▽“冷战也好 委屈也罢  不管什么时候  只要你一句软话  一个微笑或者一个拥抱  我都能笑着原谅” 

[('六集就这么看完了，伤心。特别最后一集因为总是被自己吓到结果看得断断续续，很难受了可以说是。最喜欢第二，三，六集。已经开始期待下一季了。舔一波二册美颜[污] \u200b\n', 1)],[六集 完 伤心 特别 最后 一集 总是 断断续续 难受 说 喜欢 第二 三 六集 已经 期待 一季 舔 一波 二册 美颜 污],[真实：1]，[判断：[0]]
[('喜欢的东西就去追，反感的事情就放弃，伤心了就去哭，开心就去笑。人生苦短，不要太勉强自己。 {%利川·南坪乡%} \u200b\n', 1)],[喜欢 东西 追 反感 事情 放弃 伤心 哭 开心 笑 人生 苦短 不要 太 勉强 利川 南坪 乡],[真实：1]，[判断：[0]]
[('伤心的场景。[吃惊] {%全球转播的秒拍视频%} \u200b\n', 0)],[伤心 场景 吃惊 全球 转播 秒 拍 视频],[真实：0]，[判断：[1]]
[('压抑这么久了终于在四月的最后一天放松了非常nice 超级赞离表和伤心的人别听慢歌全场嗨像是大型现场万人迪以倔强结束了天津站 可惜昨晚没有温柔[伤心]今天是五月的第一天对 五月天@五月天  \u200b\u200b\u200b{%#五月天天津#%}{%#五月天人生无限公司演唱会#%}{%#五月天天津演唱会#%} {%天津·天津奥林匹克中心%} \u200b\n', 0)],[压抑 这么久 终于 四月 最后 一天 放松 非常 nice 超级 赞离表 伤心 人别 听 慢歌 全场 嗨 像是 大型 现场 万人迪以 倔强 结束 天津站 可惜 昨晚 没有 温柔 伤心 今天 五月 第一天 五月天 五月天 天津 五月天 人生 无限公司 演唱会 五月天 天津 演唱会 天津 天津 奥林匹克 中心],[真实：0]，[判断：[1]]
[('身边总有人没有让我伤心 \u200b\n', 1)],[身边 总 有人 没有 伤心],[真实：1]，[判断：[0]]
[('我女神这么开心，但我的男神却哭😭了，我的心是开心又伤心啊@GEM鄧紫棋 @王嘉尔 {%江门·世纪广场（商圈）%} \u200b\n', 0)],[女神 开心 男神 哭 心 开心 伤心 江门 世纪 广场 商圈],[真实：0]，[判断：[1]]
[('迟到了一小时，落坐后旁边一帅哥哭的很伤心，我很想把我的肩膀借给他靠靠[污][污] \u200b\

[('那时我们有梦，关于文学，\xa0关于爱情，\xa0关于穿越世界的旅行。如今我们深夜饮酒，\xa0杯子碰到一起，\xa0都是梦破碎的声音。——北岛《波兰来客》 \u200b\n', 0)],[梦 文学 爱情 穿越 世界 旅行 如今 深夜 饮酒 杯子 碰到 一起 梦 破碎 声音 北岛 波兰 来客],[真实：0]，[判断：[1]]
[('{%#hi五月#%} 四月的温情已经遥远遥远地过去，五月的一个深夜里我做了这么一个梦。 \u200b\n', 1)],[hi 五月 四月 温情 已经 遥远 遥远 过去 五月 一个 深夜里 做 一个 梦],[真实：1]，[判断：[0]]
[('如果哪天我秀恩爱了那个人一定是世界上最好的他会牵着我的手 宠溺的摸着我的头轻声说爱我 然后脸红到脖子里而我不会再因为他一句话惆怅不已不会再在深夜抱着手机哭着睡着我想那才是我想要的爱情吧 \u200b{%#指尖艺术 情感#%} \u200b\n', 1)],[我秀 恩爱 一定 世界 最好 会 牵 手 宠 溺 摸 头 轻声 说 爱 脸红 脖子 里 不会 一句 话 惆怅 不已 不会 深夜 抱 手机 哭 睡着 想 想要 爱情 指尖 艺术 情感],[真实：1]，[判断：[0]]
[('希望以后   脾气不要给熟悉的人   也要好好爱每天深夜陪你聊天的人———网易云热评《纸短情长》 \u200b\u200b\u200b\n', 1)],[希望 以后 脾气 不要 熟悉 好好 爱 每天 深夜 陪 聊天 网易 云热评 纸短情长],[真实：1]，[判断：[0]]
[('卧槽，好恶心……[doge] {%#搞笑#%} \u200b\n', 0)],[卧槽 恶心 doge 搞笑],[真实：0]，[判断：[1]]
[('真他妈笑死了 鹦鹉＝恶心人哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈 \u200b\n', 1)],[真他妈 笑 死 鹦鹉 恶心 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈 哈哈哈],[真实：1]，[判断：[0]]
[('Qu nimei d shab wanyier ntm hai yo balfu baonimd shab fdpai laj kui wodangnis p

In [11]:
# 测试模型
test_score = nb.score(X_test,y_test)
print(test_score)

0.84


In [12]:
from sklearn import metrics
result = nb.predict(X_test)
print(metrics.classification_report(y_test, result))

             precision    recall  f1-score   support

          0       0.75      0.74      0.74       155
          1       0.88      0.89      0.88       345

avg / total       0.84      0.84      0.84       500



In [13]:
bad_cases = []
for i in range(result.shape[0]):
    if(result[i]!=y_test[i]):
        print("[%s],[真实：%s]，[判断：%s]" %(train_df['content'][i],y_test[i],result[i]))

[['静下心来', '听', '音乐书', '中', '自有', '黄金屋', '书中', '自有', '颜如玉', '岁月', '长河', '跋涉', '风光旖旎', '姹紫嫣红', '万千', '风景', '皆', '书', '中', '沐', '唐风淋', '宋雨心', '远古', '毗邻', '亦可', '同行者', '细数', '风烟', '无数', '把酒', '问青天', '豪情', '千里', '共', '婵娟', '如初', '见', '温暖', '沧海桑田', '世间', '偷天换', '地守', '一阙', '游云', '散淡', '巫山']],[真实：0]，[判断：1]
[['流星花园', '其实', '还好', '现在', '观念', '时尚', '眼光', '或许', '十几年', '之后', '现在', '剧', '会', '觉得', '搞笑', '比较', '接近', '现在', '造型', '眼光', '韩版', '花样', '男子', '流星花园', '翻拍', '看会', '之前', '重温', '韩剧', '两部', '花样', '男子', '宫', '高中', '时期', '剧是', '真的', '看看', '停不下来', '允悲']],[真实：1]，[判断：0]
[['努力', '想要', '结局', '时刻', '感觉', '似乎', '早已', '结局', '不知', '没有', '感觉', '感觉', '逃避', '否认', '口口声声', '说', '结局', '时刻', '觉得', '结局', '似乎', '没有', '重要', '会', '会', '过去', '曾经', '觉得', '难以启齿', '觉得', '毕生', '难忘', '坏', '今天', '来看', '过去', '事', '一直', '喜欢', '东西', '变成', '执念', '纠结', '坚持', '明明', '没有', '做好']],[真实：1]，[判断：0]
[['卜凡', '飞机', '惊喜', '铁血', '凡妻', '太棒了', '希望', '喜欢', '知道', '好多好多', '喜欢']],[真实：1]，[判断：0]
[['做个', '安静', '平凡', '悠闲自在', '做', '一个', '安静', '沉稳', '不会

In [14]:
# SVM 的方法
from sklearn import metrics
from sklearn import svm
# for c in [0.1,0.3,1,3,10]:
clf = svm.SVC(C=10)
clf.fit(X_train, y_train)

result = clf.predict(X_test)
print("训练集准确率:",clf.score(X_train, y_train) , "测试集准确率:", metrics.accuracy_score(y_test, result))

训练集准确率: 0.7274 测试集准确率: 0.782


KNN  训练集准确率: 0.7708 测试集准确率: 0.656

In [15]:
# knn的方法
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
result = knn.predict(X_test)
print("训练集准确率:",knn.score(X_train, y_train) , "测试集准确率:", metrics.accuracy_score(y_test, result))

训练集准确率: 0.7708 测试集准确率: 0.656


In [16]:
# 保存模型
#引入sklearn中自带的保存模块
from sklearn.externals import joblib
#保存model
joblib.dump(nb,'sklearn_save/nb.pkl')

#重新加载model，只有保存一次后才能加载model
nb2 = joblib.load('sklearn_save/nb.pkl')



FileNotFoundError: [Errno 2] No such file or directory: 'sklearn_save/nb.pkl'